In [ ]:
import gym
import numpy as np
import torch
import os
import time
import utils
import train
import buffer

env = gym.make('BipedalWalker-v3')
# env = gym.make('Pendulum-v1')
# env = gym.make('MountainCarContinuous-v0')
# env=gym.make('LunarLanderContinuous-v2')

MAX_STEPS = 2500000
MAX_BUFFER = 1000000
STATE_DIM = env.observation_space.shape[0]
ACTION_DIM = env.action_space.shape[0]
ACTION_LIM = env.action_space.high[0]

print(' The info of gym envrioment:')
print(' State Dimensions:\t', STATE_DIM)
print(' Action Dimensions:\t', ACTION_DIM)
print(' Action Limitation:\t', ACTION_LIM)

replay_buffer = buffer.MemoryBuffer(MAX_BUFFER)
trainer = train.Trainer(STATE_DIM, ACTION_DIM, ACTION_LIM, replay_buffer)

now = time.strftime("%m%d-%H%M", time.localtime(time.time()))
os.makedirs(now+'Models')

_ep = 1


observation = env.reset()
ep_reward = 0
for r in range(1,MAX_STEPS+1):
	# try:
	# 	env.render() 
	# except:
	# 	pass

	state = np.float32(observation)
	action = trainer.get_exploration_action(state)

	new_observation, reward, done, info = env.step(action)

	ep_reward += reward

	if done or r == MAX_STEPS-1:
		new_state = None
	else:
		new_state = np.float32(new_observation)
		replay_buffer.add(state, action, reward, new_state)

	observation = new_observation

	trainer.train(now)
	
	if done:
		_ep += 1
		observation = env.reset()
		with open(now+'results.txt', 'a') as f:
			f.write(f'{_ep}\t{ep_reward}\t{r}\n')
		ep_reward = 0
		

	if _ep % 10 == 0:
		trainer.save(_ep,now)


print('Training Finished!')
